In [1]:
import os
from datasets import load_from_disk

data_path = "processed_multixscience_data"

print(f"Memuat dataset dari {data_path}...")
try:
    tokenized_datasets = load_from_disk(data_path)
    print("✅ Dataset berhasil dimuat!")
    print(tokenized_datasets)
except FileNotFoundError:
    print("❌ Error: Path salah. Cek kembali lokasi folder di panel 'Input' sebelah kanan.")

print("\nContoh Kolom Dataset:", tokenized_datasets.column_names)

c:\Users\Iskandar\Documents\GitHub\Tubes-NLP-PaperMatch\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Memuat dataset dari processed_multixscience_data...
✅ Dataset berhasil dimuat!
Dataset({
    features: ['input_text', 'related_work'],
    num_rows: 30369
})

Contoh Kolom Dataset: ['input_text', 'related_work']


In [2]:
# EOS_TOKEN biasanya diperlukan agar model tahu kapan harus berhenti menulis
# Jika pakai Unsloth, biasanya sudah otomatis, tapi definisikan manual untuk aman.
EOS_TOKEN = "<|eot_id|>"

def format_prompt_llama3(examples):
    # Ambil list data dari batch
    inputs = examples["input_text"]       # Context (Abstract + Refs)
    outputs = examples["related_work"]    # Ground Truth (Target)

    prompts = []

    # System Prompt: Instruksi peran untuk AI
    system_msg = (
        "You are an academic writing assistant. "
        "Write a 'Related Work' section based on the provided text. "
        "The input contains the Current Abstract followed by References (marked with @cite_n). "
        "Synthesize these references and highlight the novelty of the Current Abstract."
    )

    for input_text, output_text in zip(inputs, outputs):
        # Struktur Llama 3 Instruct Resmi
        text = f"""<|begin_of_text|><|start_header_id|>system<|end_header_id|>

{system_msg}<|eot_id|><|start_header_id|>user<|end_header_id|>

{input_text}<|eot_id|><|start_header_id|>assistant<|end_header_id|>

{output_text}{EOS_TOKEN}"""

        prompts.append(text)

    # Kembalikan dalam kolom baru bernama 'text' (biasanya SFTTrainer mencari kolom ini)
    return { "text": prompts }

# --- CARA PAKAI ---
# Asumsi 'tokenized_datasets' adalah HuggingFace Dataset yang sudah Anda load
# Terapkan fungsi format_prompt_llama3 ke seluruh dataset
formatted_datasets = tokenized_datasets.map(format_prompt_llama3, batched=True)
# print(formatted_datasets['text'][0])

In [ ]:
from unsloth import FastLanguageModel
import torch
if not hasattr(torch._inductor, "config"):
    class DummyConfig:
        pass
    torch._inductor.config = DummyConfig()


max_seq_length = 1024 # Sesuai hasil EDA Anda
dtype = None # Auto detection
load_in_4bit = True # WAJIB True untuk Kaggle

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/llama-3-8b-bnb-4bit", # Versi 4-bit yang ringan
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

# target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
#                       "gate_proj", "up_proj", "down_proj",]
target_modules = ["q_proj", "v_proj"]

# Tambahkan Adapter LoRA (Agar model bisa belajar)
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Rank
    target_modules = target_modules,
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth", # Hemat VRAM
    random_state = 3407,
)

print("Model Llama 3 8B (4-bit) siap di-train di Kaggle!")

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


AttributeError: module 'torch._inductor' has no attribute 'config'

In [ ]:
import torch
print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No GPU")

True
NVIDIA GeForce RTX 3060 Laptop GPU
2.4.1+cu121
2.4.1+cu121
True
